<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Downloading-HRVfrom-Drive" data-toc-modified-id="Downloading-HRVfrom-Drive-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Downloading HRVfrom Drive</a></span></li></ul></div>

In [24]:
## files and direcotries
import os
import pandas as pd
kdd_dir = 'path'
file_path = '/home/jfrasern/code/Jackfn96/auto evaluation'

files = [f[:-4] for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]
files.sort()
print(len(files))

39


In [25]:
# from os.path import dirname, abspath
# d = os.path.join(os.path.dirname(os.getcwd()), 'raw_data')

# files = [f[:-4] for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]
# files.sort()
# print(len(files))
os.listdir(file_path)

['kss_2021-10-22 12-35-46..csv',
 'kss_2021-10-28 15-04-13 g..csv',
 'kss_2021-11-24 11-01-03 eca..csv',
 'annotation_kss_2021-11-17 10-58-38 e99.xlsx',
 '2021-11-26 15-16-28 ea9 (1).xlsx',
 'kss_2021-12-07 14-35-11 ea9 (1)..csv',
 'kss_2021-11-22 11-26-24 ea9..csv',
 '2021-10-25 10-31-58 c27.xlsx',
 '2021-11-29 13-48-17 eca.xlsx',
 'kss_2021-11-02 15-23-39 g..csv',
 'kss_2022-01-05 15-25-18..csv',
 'kss_2021-11-26 12-34-32 1c3..csv',
 '2021-11-05 13-59-29 ea9.xlsx',
 'annotation_kss_2021-11-24 15-36-04 e99.xlsx',
 'kss_2021-10-26 13-32-50..csv',
 '2022-01-07 14-02-27.xlsx',
 'kss_2021-11-29 13-48-17 eca..csv',
 'kss_2021-11-05 13-59-29 ea9..csv',
 'kss_2021-11-30 16-16-03..csv',
 '2021-11-30 16-16-03.xlsx',
 '2021-11-22 11-26-24 ea9.xlsx',
 '2021-11-04 11-01-00.xlsx',
 '2021-11-02 15-23-39 g.xlsx',
 '2021-10-28 15-04-13 g.xlsx',
 'kss_2021-11-04 11-01-00..csv',
 '2021-10-22 12-35-46.xlsx',
 '2021-11-26 12-34-32 1c3.xlsx',
 '2021-10-26 13-32-50.xlsx',
 'kss_2021-11-26 15-16-28 ea9 (1).

In [26]:
from os.path import dirname, abspath
d = os.path.join(os.path.dirname(os.getcwd()))
parent = os.path.abspath(os.path.join(d, '../'))
root_dir = os.path.join(parent, 'auto evaluation/')
root_dir

'/home/jfrasern/code/Jackfn96/auto evaluation/'

In [27]:
!pip install openpyxl

You should consider upgrading via the '/home/jfrasern/.pyenv/versions/3.8.12/envs/3DS/bin/python -m pip install --upgrade pip' command.


In [28]:
df = pd.read_excel(f'{root_dir}{files[0]}xlsx') # temporary df to load kss file
df = df.drop(columns = ['Unnamed: 0'], inplace=True, axis=1)

df

In [29]:
df

In [30]:
for file_name in files:
    print(file_name)

2021-10-22 12-35-46.
2021-10-25 10-31-58 c27.
2021-10-26 13-32-50.
2021-10-28 15-04-13 g.
2021-11-02 15-23-39 g.
2021-11-04 11-01-00.
2021-11-05 13-59-29 ea9.
2021-11-19 13-38-33.
2021-11-22 11-26-24 ea9.
2021-11-24 11-01-03 eca.
2021-11-26 12-34-32 1c3.
2021-11-26 15-16-28 ea9 (1).
2021-11-29 13-48-17 eca.
2021-11-30 16-16-03.
2021-12-07 14-35-11 ea9 (1).
2022-01-05 15-25-18.
2022-01-07 14-02-27.
annotation_kss (1) PIERRE BOIVIN.
annotation_kss_2021-11-17 10-58-38 e99.
annotation_kss_2021-11-18 13-21-41 e99.
annotation_kss_2021-11-24 15-36-04 e99.
annotation_kss_2021-12-01 13-20-00 e99.
kss_2021-10-22 12-35-46.
kss_2021-10-25 10-31-58 c27.
kss_2021-10-26 13-32-50.
kss_2021-10-28 15-04-13 g.
kss_2021-11-02 15-23-39 g.
kss_2021-11-04 11-01-00.
kss_2021-11-05 13-59-29 ea9.
kss_2021-11-19 13-38-33.
kss_2021-11-22 11-26-24 ea9.
kss_2021-11-24 11-01-03 eca.
kss_2021-11-26 12-34-32 1c3.
kss_2021-11-26 15-16-28 ea9 (1).
kss_2021-11-29 13-48-17 eca.
kss_2021-11-30 16-16-03.
kss_2021-12-07 14-3

In [31]:
len(files)

39

In [32]:
## parameters
kssdfs = {}
df_num = 0

import numpy as np
fps = 1/2 # 
for file_name in files:
#     video_annot_df = pd.read_csv(f'{root_dir}{file_name}.mp4.csv', sep=';')
#     video_annot_df.drop('Unnamed: 0', inplace=True, axis=1)
#     video_annot_df['exp_id'] = file_name
#     video_annot_df = timestamp_formatting(video_annot_df, 'timestamp_lena')
#     video_annot_df['event_onset'] = video_annot_df['Instant']/1000
#     video_annot_df['event_end'] = video_annot_df['event_onset'] + video_annot_df['duree']
#     print('the shape of video annotation dataframe is', video_annot_df.shape)
    ## kss dataframe
    df = pd.read_excel(f'{root_dir}{file_name}xlsx') # temporary df to load kss file
    df.drop(columns = ['Unnamed: 0'], inplace=True, axis=1)
    video_dur = df.iloc[4, :].values
    print(f'{file_name} : {video_dur}')
    column_names = df.iloc[6, :].values
    kss_raw_df = pd.DataFrame(df.iloc[7:, :].values, columns = column_names)
    kss_raw_df.drop('Buffer', inplace = True, axis=1)
    print('the shape of kss data frame is', kss_raw_df.shape)
    print(df_num)
    ## correction of kss_raw_df
    frame_numbers = int(video_dur[1])*60 + int(video_dur[2])
    onset= int(kss_raw_df.iloc[0,0].split('-')[0].split(':')[1])
    ## alternatively if you don't have the interval columns, you can use 
    # hand_sign = df.iloc[3, :].values
    # onset =  int(hand_sign[1])*60 + int(hand_sign[2])
    
    ## kss_interval is probably constant for all experiments ~ 300s
    kss_interval = int(kss_raw_df.iloc[0,0].split('-')[1].split(':')[0])*60
    
    
    video_timing_df = pd.DataFrame.from_records([{'duration': frame_numbers, 
                                                  'kss_onset':onset,
                                                  'kss_interval': kss_interval,
                                                  'fps': fps}])
    ## from here on you should repeat the kss accoring to the sampling rate of HRV recording,
    ## I had 2frames per seconds 
    recording_time = np.arange(0, frame_numbers, fps).tolist() 
    
    ## populating a dataframe according to the fps with corresponding kss values
    kss_impute = np.zeros(len(recording_time))
    interval = int(kss_interval / fps)
    for i in range(kss_raw_df.shape[0]):
        kss_impute[i*interval+onset: (i+1)*interval+onset] = kss_raw_df.iloc[i, 1]
    # x = downsample_to_proportion(kss_impute, .03333)
#     plt.plot(kss_impute)
    
    ## merging the kss annotation with frames 
    kss_annotations = {'time_points': recording_time, 'kss_value': kss_impute }
    kss_annotations_df = pd.DataFrame.from_dict(kss_annotations)

    kssdfs[file_name.split(' ')[0]] = kss_annotations_df
    df_num+=1
    ## saving
#     kss_annotations_df.to_csv(f'{root_dir}kss_{file_name}.csv')


2021-10-22 12-35-46. : ['Video duration' 62 5]
the shape of kss data frame is (13, 2)
0
2021-10-25 10-31-58 c27. : ['Video duration' 43 23]
the shape of kss data frame is (9, 2)
1
2021-10-26 13-32-50. : ['Video duration' 70 7]
the shape of kss data frame is (14, 2)
2
2021-10-28 15-04-13 g. : ['Video duration' 68 24]
the shape of kss data frame is (14, 2)
3
2021-11-02 15-23-39 g. : ['Video duration' 67 38]
the shape of kss data frame is (14, 2)
4
2021-11-04 11-01-00. : ['Video duration' 69 23]
the shape of kss data frame is (14, 2)
5
2021-11-05 13-59-29 ea9. : ['Video duration' 66 17]
the shape of kss data frame is (14, 2)
6
2021-11-19 13-38-33. : ['Video duration' 67 33]
the shape of kss data frame is (14, 2)
7
2021-11-22 11-26-24 ea9. : ['Video duration' 66 27]
the shape of kss data frame is (14, 2)
8
2021-11-24 11-01-03 eca. : ['Video duration' 66 54]
the shape of kss data frame is (14, 2)
9
2021-11-26 12-34-32 1c3. : ['Video duration' 67 16]
the shape of kss data frame is (14, 2)
10

ValueError: cannot convert float NaN to integer

In [33]:
ans = (66 * 60 + 28) * 2
ans

7976

In [34]:
kssdfs['2021-10-22']

,time_points,kss_value
0,0.0,0.0
1,0.5,0.0
2,1.0,0.0
3,1.5,0.0
4,2.0,0.0
...,...,...
7445,3722.5,3.0
7446,3723.0,3.0
7447,3723.5,3.0
7448,3724.0,3.0


# Downloading HRVfrom Drive

In [35]:
%load_ext autoreload
%autoreload 2

In [36]:
drive = Drive()

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [20]:
from DDDS.hrv import HRV
hrv = HRV()

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [ ]:
dataframes = hrv.get_dataframes()